In [1]:
## This file implements neural networks and random forest on pyopresabs_qual.
## Due to the imbalanced data and limited cases for class 2, we implement the over-sampling method.
## For fully-connected neural networks, the accuracy is 96.95% for over-sampling data, and 96.45% after regularization.
## For random forest, the accuracy is 98.48% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 96.06% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/pyopresabs_qual.csv')
df.shape

(255, 539)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     0
27     0
28     0
29     0
      ..
225    0
226    0
227    1
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

0    218
1     33
2      4
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8666,group_8667,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,109,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 538)

In [10]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8666,group_8667,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 537) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218), (2, 218)]


Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
############# Fully-Connected Neural Network ################

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [16]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [17]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 412us/step - loss: 0.6470 - accuracy: 0.7505 - val_loss: 0.4950 - val_accuracy: 0.8173
Epoch 2/100
457/457 [==============================] - 0s 99us/step - loss: 0.4078 - accuracy: 0.8818 - val_loss: 0.3802 - val_accuracy: 0.8680
Epoch 3/100
457/457 [==============================] - 0s 96us/step - loss: 0.3065 - accuracy: 0.8993 - val_loss: 0.3058 - val_accuracy: 0.8832
Epoch 4/100
457/457 [==============================] - 0s 92us/step - loss: 0.2458 - accuracy: 0.9278 - val_loss: 0.2583 - val_accuracy: 0.9340
Epoch 5/100
457/457 [==============================] - 0s 124us/step - loss: 0.1991 - accuracy: 0.9453 - val_loss: 0.2204 - val_accuracy: 0.9391
Epoch 6/100
457/457 [==============================] - 0s 140us/step - loss: 0.1660 - accuracy: 0.9519 - val_loss: 0.1993 - val_accuracy: 0.9594
Epoch 7/100
457/457 [==============================] - 0s 77us/step - loss: 0.1400 - ac

457/457 [==============================] - 0s 62us/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0946 - val_accuracy: 0.9746
Epoch 58/100
457/457 [==============================] - 0s 62us/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0954 - val_accuracy: 0.9746
Epoch 59/100
457/457 [==============================] - 0s 60us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0943 - val_accuracy: 0.9746
Epoch 60/100
457/457 [==============================] - 0s 60us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0975 - val_accuracy: 0.9746
Epoch 61/100
457/457 [==============================] - 0s 70us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0967 - val_accuracy: 0.9746
Epoch 62/100
457/457 [==============================] - 0s 113us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0983 - val_accuracy: 0.9746
Epoch 63/100
457/457 [==============================] - 0s 129us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0974 - val_accuracy: 0.9

In [19]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

197/197 [==============================] - 0s 59us/step
over-sampling test accuracy: 96.95%


In [20]:
#### add dropout and regularizor
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [21]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 656us/step - loss: 3.8808 - accuracy: 0.5689 - val_loss: 0.8186 - val_accuracy: 0.7310
Epoch 2/100
457/457 [==============================] - 0s 92us/step - loss: 3.3015 - accuracy: 0.6696 - val_loss: 0.6853 - val_accuracy: 0.8325
Epoch 3/100
457/457 [==============================] - 0s 75us/step - loss: 3.2504 - accuracy: 0.6893 - val_loss: 0.6841 - val_accuracy: 0.8426
Epoch 4/100
457/457 [==============================] - 0s 71us/step - loss: 3.2569 - accuracy: 0.7046 - val_loss: 0.6801 - val_accuracy: 0.8680
Epoch 5/100
457/457 [==============================] - 0s 81us/step - loss: 2.9134 - accuracy: 0.7155 - val_loss: 0.7047 - val_accuracy: 0.8528
Epoch 6/100
457/457 [==============================] - 0s 69us/step - loss: 3.1634 - accuracy: 0.6805 - val_loss: 0.7667 - val_accuracy: 0.8731
Epoch 7/100
457/457 [==============================] - 0s 65us/step - loss: 2.3652 - accu

457/457 [==============================] - 0s 66us/step - loss: 0.8440 - accuracy: 0.8271 - val_loss: 0.6550 - val_accuracy: 0.9695
Epoch 58/100
457/457 [==============================] - 0s 64us/step - loss: 0.8409 - accuracy: 0.7987 - val_loss: 0.6558 - val_accuracy: 0.9695
Epoch 59/100
457/457 [==============================] - 0s 64us/step - loss: 0.8934 - accuracy: 0.7965 - val_loss: 0.6937 - val_accuracy: 0.9645
Epoch 60/100
457/457 [==============================] - 0s 70us/step - loss: 0.7814 - accuracy: 0.8031 - val_loss: 0.6527 - val_accuracy: 0.9695
Epoch 61/100
457/457 [==============================] - 0s 73us/step - loss: 0.7805 - accuracy: 0.8162 - val_loss: 0.6678 - val_accuracy: 0.9695
Epoch 62/100
457/457 [==============================] - 0s 76us/step - loss: 0.8556 - accuracy: 0.7921 - val_loss: 0.6524 - val_accuracy: 0.9695
Epoch 63/100
457/457 [==============================] - 0s 87us/step - loss: 0.6874 - accuracy: 0.8206 - val_loss: 0.6396 - val_accuracy: 0.969

In [23]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

197/197 [==============================] - 0s 70us/step
over-sampling test accuracy: 96.45%


In [24]:
############## Random Forest ##############

In [25]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [26]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 98.48%


In [27]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.96774194 0.95698925 0.97802198 0.94444444 0.95555556]
0.9605506321635355
